In [1]:
# default_exp to_sagemaker

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# export


import os
from pathlib import Path
from typing import Any, Dict, Iterable

from nbdev.export import find_default_export, get_config, read_nb

from sciflow.data_handler import extract_param_meta
from sciflow.params import params_as_dict
from sciflow.parse_module import FuncDetails, extract_steps
from sciflow.utils import prepare_env

In [7]:
from sciflow.to_metaflow import titleize, extract_module_only, get_module_name, get_flow_path

# Sciflow Notebook to Sagemaker Pipeline

> Converts from a `sciflow` format notebook to a `sagemaker` pipeline. 

Currently Supported features:

* Linear/sequential DAGs
* Simple `Parameters`

In [8]:
nb_path = Path(os.path.join("test", "test_clustering.ipynb"))
nb = read_nb(nb_path)
module_name = find_default_export(nb["cells"]).replace(".", "/")
test_module = os.path.join(get_config().path("lib_path"), f"{module_name}.py")

In [9]:
flow_path = get_flow_path(nb_path, "sagemaker")

In [10]:
flow_path

Path('/home/sagemaker-user/git/sciflow/nbs/test/flows/sagemaker/test_clustering.py')

In [11]:
steps = extract_steps(test_module)

In [31]:
# export


def nb_to_sagemaker_pipeline(nb_path: Path, flow_path: Path, silent=True, track_experiment=True):
    nb = read_nb(nb_path)
    lib_name = get_config().get("lib_name")
    module_name = find_default_export(nb["cells"])
    if not module_name:
        return
    module_name = module_name
    path_sep_module_name = module_name.replace(".", "/")
    nb_name = os.path.basename(nb_path)
    exported_module = os.path.join(
        get_config().path("lib_path"), f"{path_sep_module_name}.py"
    )
    steps = extract_steps(exported_module)
    if len(steps) == 0:
        print('Skipping sagemaker conversion - not steps found')
        return
    params = params_as_dict(nb_path)
    if len(params) == 0:
        print(f"No params cell found for: {os.path.basename(nb_path)}")
    pipeline_class_name = f"{titleize(extract_module_only(module_name))}Pipeline"
    write_pipeline_to_files(
        flow_path,
        pipeline_class_name,
        lib_name,
        module_name,
        steps,
        params,
        track_experiment,
    )
    if not silent:
        print(
            f"Converted {nb_name} to {pipeline_class_name} in: {os.path.basename(flow_path)}"
        )

In [32]:
#  export


def write_pipeline_to_files(
    flow_path: Path,
    pipeline_class_name: str,
    lib_name: str,
    module_name: str,
    steps: Iterable[FuncDetails],
    params: dict,
    track_experiment: bool,
):
    if not os.path.exists(flow_path.parent):
        os.mkdir(flow_path.parent)
    fq_module_name = f"{lib_name}.{module_name}"
    param_meta = extract_param_meta(fq_module_name, params)
    with open(flow_path, "w") as flow_file:
        flow_file.write("#!/usr/bin/env python\n")
        flow_file.write("# coding=utf-8\n")
        flow_file.write("# SCIFLOW GENERATED FILE - EDIT COMPANION NOTEBOOK\n")
            
        flow_file.write("import os\n")
        
        flow_file.write("import sagemaker\n")
        flow_file.write("from sagemaker.session import Session\n")
        flow_file.write("from sagemaker.workflow.pipeline import Pipeline\n")
        
        has_train_step = any([s.name.startswith(n) for n in['fit', 'train'] for s in steps])
        has_processing_step = sum([s.name.startswith(n) for n in['fit', 'train'] for s in steps]) != len(steps)
        
        if has_train_step and has_processing_step:
            flow_file.write("from sagemaker.workflow.steps import ProcessingStep, TrainingStep\n")
        if has_processing_step:
            flow_file.write("from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput\n")
            flow_file.write("from sagemaker.workflow.pipeline import Pipeline\n")
        if has_train_step:
            flow_file.write("from sagemaker.inputs import TrainingInput\n")
            flow_file.write("from sagemaker.estimator import Estimator\n")
                  
        has_sm_param = any((p.has_sagemaker_param for p in param_meta.values()))
        if has_sm_param:
            instance_types = [p.instance_type for p in param_meta.values()]
            sm_params_import = "from sagemaker.workflow.parameters import "
            if int in instance_types:
                sm_params_import += ", ParameterInteger"
            if float in instance_types:
                sm_params_import += ", ParameterFloat"
            if str in instance_types:
                sm_params_import += ", ParameterString"

            flow_file.write(sm_params_import + "\n")
        
        flow_file.write("\n")
        flow_file.write(f"from {fq_module_name} import {', '.join([s.name for s in steps])}\n")
        if len(params) > 0:
            flow_file.write(
                f"from {fq_module_name} import {', '.join(params.keys())}\n"
            )

        flow_file.write(f"\n\nclass {pipeline_class_name}():\n")
        single_indent = "    "
        write_params(flow_file, param_meta, single_indent)
        flow_file.write("\n")
        write_steps(
            fq_module_name,
            flow_file,
            steps,
            param_meta,
            single_indent,
            track_experiment,
        )
        flow_file.write("\n")

        flow_file.write('if __name__ == "__main__":\n')
        flow_file.write(f"{single_indent}{pipeline_class_name}()")

In [33]:
# export


def write_observers(lib_name, flow_file, module_name, bucket_name, project):
    pass

In [34]:
# export


def write_track_flow(flow_file, track_experiment):
    pass

In [35]:
#  export


def write_params(flow_file, param_meta, single_indent):
    for param in param_meta.keys():
        if param_meta[param].instance_type == int:
            flow_file.write(
                f"{single_indent}{param} = ParameterInteger(name='{param}', default_value={param})\n"
            )
        elif param_meta[param].instance_type == float:
            flow_file.write(
                f"{single_indent}{param} = ParameterFloat(name='{param}', default_value={param})\n"
            )
        elif param_meta[param].instance_type == str:
            flow_file.write(
                f"{single_indent}{param} = ParameterString(name='{param}', default_value={param})\n"
            )

In [57]:
def write_script_processor(flow_file):
    flow_file.write("""
        script_processor = ScriptProcessor(
                command=['python3'],
                image_uri="141502667606.dkr.ecr.eu-west-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3",
                role=self.role,
                instance_count=1,
                instance_type="ml.m5.xlarge",
                sagemaker_session=self.sagemaker_session,
                env={'AWS_DEFAULT_REGION': self.region},
                base_job_name=f'processing-job/{__file__}'
        )
        """
)

In [61]:
# export


def format_arg(arg, param_meta):
    if arg in param_meta and not param_meta[arg].has_metaflow_param:
        result = arg
    else:
        result = "self." + arg
    return result


def write_steps(
    fq_module_name, flow_file, steps, param_meta, single_indent, track_experiment
):
    step_names = [s.name for s in steps]
    for i, step in enumerate(steps):
        flow_file.write(f"{single_indent}def {step.name}(self):\n")
        if step.docstring:
            flow_file.write(f"{indent_multiline(step.docstring, 2)}\n")
        # Processing step
        if not any([step.name.startswith(n) for n in ('fit', 'train')]):
            write_script_processor(flow_file)
            
            flow_file.write("\n")
            flow_file.write(f"{single_indent}{single_indent}start_step = ProcessingStep(\n")
            flow_file.write(f"{single_indent}{single_indent}{single_indent}name={step.name}\n")
            flow_file.write(f"{single_indent}{single_indent}{single_indent}processor = script_processor\n")
            flow_file.write(f"{single_indent}{single_indent}{single_indent}code = {fq_module_name}_{step.name}.py\n") 
            flow_file.write(f"{single_indent}{single_indent})\n")
        else:
            # Trainign step
            pass
        flow_file.write("\n")

In [62]:
# export


def write_track_capture(flow_file):
    flow_file.write(
        f"""
        for key in results.keys():
            if key in self.__dict__:
                self.__dict__[key] = self.__dict__[key] + results[key]
            else:
                self.__dict__[key] = results[key]

"""
    )

In [63]:
nb_to_sagemaker_pipeline(nb_path, flow_path, silent=False, track_experiment=False)

Converted test_clustering.ipynb to TestClusteringPipeline in: test_clustering.py
